In [1]:
import pandas as pd
import numpy as np
from isoweek import Week

In [2]:
data = pd.read_csv('./pro_actual_sales.csv')
data.head(5)

,regionSeg1,productSeg1,productSeg2,regionSeg2,regionSeg3,productSeg3,yearweek,year,week,qty
0,A01,NaN,PG05,SALESID0001,SITEID0004,ITEM0445,201306,2013,6,5
1,A01,NaN,PG05,SALESID0001,SITEID0004,ITEM0445,201307,2013,7,8
2,A01,NaN,PG05,SALESID0001,SITEID0004,ITEM0445,201308,2013,8,7
3,A01,NaN,PG05,SALESID0001,SITEID0004,ITEM0445,201309,2013,9,1
4,A01,NaN,PG05,SALESID0001,SITEID0004,ITEM0445,201310,2013,10,1


In [3]:
refineData = data[['regionSeg1','productSeg2','regionSeg2','regionSeg3','productSeg3','yearweek','year','week','qty']]

In [4]:
refineData.head(5)

,regionSeg1,productSeg2,regionSeg2,regionSeg3,productSeg3,yearweek,year,week,qty
0,A01,PG05,SALESID0001,SITEID0004,ITEM0445,201306,2013,6,5
1,A01,PG05,SALESID0001,SITEID0004,ITEM0445,201307,2013,7,8
2,A01,PG05,SALESID0001,SITEID0004,ITEM0445,201308,2013,8,7
3,A01,PG05,SALESID0001,SITEID0004,ITEM0445,201309,2013,9,1
4,A01,PG05,SALESID0001,SITEID0004,ITEM0445,201310,2013,10,1


In [5]:
promotionData = pd.read_csv('./pro_promotion.csv')

In [6]:
##targetweek < planweek 데이터 제거
planweekMin = min(promotionData.planwee)

In [7]:
testData = promotionData[promotionData['targetweek'] >= planweekMin]
testData = testData.sort_values(['targetweek'],axis=0)
testData.head()


,regionseg,salesid,productgroup,item,targetweek,planwee,map_price,ir,pmap,pmap10,pro_percent
839,A01,SALESID0001,PG04,ITEM0306,201601,201601,3599,378,3221,2899,0.194498
6537,A01,SALESID0001,PG05,ITEM0034,201601,201601,1099,318,781,703,0.360328
12728,A01,SALESID0001,PG05,ITEM0457,201601,201601,1599,262,1337,1203,0.247655
3774,A01,SALESID0001,PG04,ITEM0271,201601,201601,1999,222,1777,1599,0.200100
9168,A01,SALESID0001,PG05,ITEM0095,201601,201601,1099,262,837,753,0.314832


In [8]:
mergeData = pd.merge(refineData,testData,how="left",left_on=["regionSeg1","productSeg2","regionSeg2","productSeg3","yearweek"],right_on=["regionseg","productgroup","salesid","item","targetweek"])\

joinData = mergeData.dropna()

In [9]:
joinData = joinData[['regionSeg1','productSeg2','regionSeg2','regionSeg3','productSeg3','yearweek','year','week','qty','map_price','ir','pmap','pmap10','pro_percent']]
joinData = joinData.sort_values(['productSeg2','yearweek','regionSeg3','productSeg3'])
joinData.head()

,regionSeg1,productSeg2,regionSeg2,regionSeg3,productSeg3,yearweek,year,week,qty,map_price,ir,pmap,pmap10,pro_percent
135415,A01,PG01,SALESID0001,SITEID0001,ITEM0110,201601,2016,1,32,699.0,89.0,610.0,549.0,0.214592
135556,A01,PG01,SALESID0001,SITEID0001,ITEM0111,201601,2016,1,1,649.0,39.0,610.0,549.0,0.154083
135887,A01,PG01,SALESID0001,SITEID0001,ITEM0116,201601,2016,1,15,549.0,50.0,499.0,449.0,0.182149
136057,A01,PG01,SALESID0001,SITEID0001,ITEM0117,201601,2016,1,149,599.0,100.0,499.0,449.0,0.250417
133688,A01,PG01,SALESID0001,SITEID0001,ITEM0118,201601,2016,1,15,549.0,50.0,499.0,449.0,0.182149


In [10]:
##qty빠진 주차 0값으로 채워넣기
def postWeek(inputYW, gapWeek) :
    inputYW = str(inputYW)
    inputY = inputYW[0:4]
    inputW = inputYW[4:]
    
    gapWeek = int(gapWeek)
    
    sumWeek = int(inputW) + gapWeek
    
    currYear = int(inputY)
    lastWeek = Week.last_week_of_year(currYear).week
    
    while(sumWeek > lastWeek) :
        sumWeek -= lastWeek
        currYear += 1
        lastWeek = Week.last_week_of_year(currYear).week
        
    if(sumWeek < 10) :
        lastAnswer = str(currYear) + "0" + str(sumWeek)
    else :
        lastAnswer = str(currYear) + str(sumWeek)
    
    return lastAnswer

In [11]:
yearweekmin = str(min(joinData.yearweek))
yearweekmax = str(max(joinData.yearweek))

i=1
tempArray = [yearweekmin]
print(tempArray)
while(i < 27):
    tempArray.append(postWeek(yearweekmin,i))
    i = i+1

['201601']


In [12]:
yearweektemp = []
yearweekData = joinData.yearweek
test = str(set(yearweekData))
tempArray = set(tempArray)

In [17]:
joinData["NEW_QTY"] = np.where(joinData["qty"] < 0, 0, joinData["qty"])

In [16]:
##이동평균함수 구하기
joinData["MA"] = joinData.qty.rolling(window=5,center=True,min_periods=1).mean()

In [30]:
groupKey = ["productSeg2","regionSeg3","yearweek"]
groupData = joinData.groupby(groupKey)[["NEW_QTY"]].mean()
groupDataResult = groupData.reset_index()
tempData = joinData
mergeData = pd.merge(tempData,groupDataResult,how="left",on=groupKey)
mergeData.head()

,regionSeg1,productSeg2,regionSeg2,regionSeg3,productSeg3,yearweek,year,week,qty,map_price,ir,pmap,pmap10,pro_percent,MA,NEW_QTY_x,NEW_QTY_y
0,A01,PG01,SALESID0001,SITEID0001,ITEM0110,201601,2016,1,32,699.0,89.0,610.0,549.0,0.214592,16.00,32,29.5
1,A01,PG01,SALESID0001,SITEID0001,ITEM0111,201601,2016,1,1,649.0,39.0,610.0,549.0,0.154083,49.25,1,29.5
2,A01,PG01,SALESID0001,SITEID0001,ITEM0116,201601,2016,1,15,549.0,50.0,499.0,449.0,0.182149,42.40,15,29.5
3,A01,PG01,SALESID0001,SITEID0001,ITEM0117,201601,2016,1,149,599.0,100.0,499.0,449.0,0.250417,39.40,149,29.5
4,A01,PG01,SALESID0001,SITEID0001,ITEM0118,201601,2016,1,15,549.0,50.0,499.0,449.0,0.182149,40.20,15,29.5


In [34]:
def groupRolling(oneGroup):

    #len(list(groupDataSet.groups))

    #oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[2])
    #oneGroup.head(1)

    indexGroupData = oneGroup.reset_index(drop=True)
    indexGroupData["MA2"] = indexGroupData["NEW_QTY_x"].rolling(window=5,center=True,min_periods=1).mean()

    return indexGroupData

In [35]:
finalResult = mergeData.groupby(groupKey).apply(groupRolling)
finalResult.head()

regionSeg1 productSeg2   regionSeg2  \
productSeg2 regionSeg3 yearweek                                         
PG01        SITEID0001 201601   0        A01        PG01  SALESID0001   
                                1        A01        PG01  SALESID0001   
                                2        A01        PG01  SALESID0001   
                                3        A01        PG01  SALESID0001   
                                4        A01        PG01  SALESID0001   

                                   regionSeg3 productSeg3  yearweek  year  \
productSeg2 regionSeg3 yearweek                                             
PG01        SITEID0001 201601   0  SITEID0001    ITEM0110    201601  2016   
                                1  SITEID0001    ITEM0111    201601  2016   
                                2  SITEID0001    ITEM0116    201601  2016   
                                3  SITEID0001    ITEM0117    201601  2016   
                                4  SITEID0001    ITEM0118    201601  2016   

                                   week  qty  map_price     ir   pmap  pmap10  \
productSeg2 regionSeg3 yearweek                                                 
PG01        SITEID0001 201601   0     1   32      699.0   89.0  610.0   549.0   
                                1     1    1      649.0   39.0  610.0   549.0   
                                2     1   15      549.0   50.0  499.0   449.0   
                                3     1  149      599.0  100.0  499.0   449.0   
                                4     1   15      549.0   50.0  499.0   449.0   

                                   pro_percent     MA  NEW_QTY_x  NEW_QTY_y  \
productSeg2 regionSeg3 yearweek                                               
PG01        SITEID0001 201601   0     0.214592  16.00         32       29.5   
                                1     0.154083  49.25          1       29.5   
                                2     0.182149  42.40         15       29.5   
                                3     0.250417  39.40        149       29.5   
                                4     0.182149  40.20         15       29.5   

                                     MA2  
productSeg2 regionSeg3 yearweek           
PG01        SITEID0001 201601   0  16.00  
                                1  49.25  
                                2  42.40  
                                3  39.40  
                                4  40.20

In [33]:
finalResult = finalResult.reset_index(drop=True)

In [ ]:
##계절성 지수 산출
finalResult["SEASONALITY"] = finalResult["qty_y"] / finalResult["MA2"]